### CSV parsing into frozen dataclass

In [8]:
from src.data_loader import load_market_data

mkt_data = load_market_data("market_data.csv")
print(mkt_data[0])

try:
    mkt_data[0].price = 1
except Exception as e:
    print("ERROR! Cannot update MarketDataPoint.price", e)

MarketDataPoint(timestamp=datetime.datetime(2025, 9, 19, 15, 45, 57, 810291), symbol='AAPL', price=152.92)
ERROR! Cannot update MarketDataPoint.price cannot assign to field 'price'


### Mutable behavior of Order

In [8]:
# Demonstrate in a unit test that you can update Order.status but not MarketDataPoint.price.
from src.models import MarketDataPoint, Order
import datetime

# Immutable
mkt_data_pt = MarketDataPoint(datetime.datetime(2025,9,19,9,30), 'AAPL', 150.0)
try:
    mkt_data_pt.price = 200
except Exception as e:
    print("ERROR! Cannot update MarketDataPoint.price:", e)

# Mutable
o = Order('AAPL', 10, 150.0, 'NEW')
print("Original", o)
o.status = 'FILLED'
print("Updated", o)

ERROR! Cannot update MarketDataPoint.price: cannot assign to field 'price'
Original <src.models.Order object at 0x1084528d0>
Updated <src.models.Order object at 0x1084528d0>


### Exception raising and handling

In [12]:
import random
from src.models import *
from src.engine import *

engine = ExecutionEngine()

# OrderError
class BadStrategy:
    def generate_signals(self, tick):
        return [('BUY', 'AAPL', -10, 150.0)]  # invalid qty
engine.process([MarketDataPoint(datetime.datetime.now(), "AAPL", 150.0)], [BadStrategy()])

# ExecutionError
class GoodStrategy:
    def generate_signals(self, tick):
        return [('BUY', 'AAPL', 10, 150.0)]  # valid order

_random = random.random    
random.random = lambda: 0.0
engine.process([MarketDataPoint(datetime.datetime.now(), "AAPL", 150.0)], [GoodStrategy()])
random.random = _random

print("Error log contents:", engine.error_log)

Error log contents: ['Order Error! Invalid quantity: -10', 'Execution Error! Order execution failed']
